In [2]:
!pip install elasticsearch

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import csv
from collections import deque
import elasticsearch
from elasticsearch import helpers
import re


In [3]:
def readMovies():
    csvfile = open('ml-latest-small/movies.csv', 'r', encoding='utf-8', errors='ignore')
    
    reader = csv.DictReader( csvfile )
    for line in reader:
        movie = {}
        title = re.sub(" \(.*\)$", "", re.sub('"','', line['title']))
        year = line['title'][-5:-1]
        if (not year.isdigit()):
            year = "2016"
        genres = line['genres'].split('|')
        movie['movieId'] = int(line['movieId'])
        movie['title'] = title
        movie['year'] = int(year)
        movie['genres'] = genres
        yield movie

In [4]:
es = elasticsearch.Elasticsearch(["http://199.241.138.138:9200"], http_auth=('elastic', 'changeme'))

es.indices.delete(index="movies",ignore=404)
deque(helpers.parallel_bulk(es,readMovies(),index="movies"), maxlen=0)
es.indices.refresh()

<ipython-input-4-6110a9a13a08>:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = elasticsearch.Elasticsearch(["http://199.241.138.138:9200"], http_auth=('elastic', 'changeme'))
<ipython-input-4-6110a9a13a08>:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="movies",ignore=404)


ObjectApiResponse({'_shards': {'total': 30, 'successful': 20, 'failed': 0}})